# <font color="hotpink">Eco_Fertilization</font>

## BestTimeToFertilize Module
* Using <a href="https://www.weatherbit.io/api/weather-forecast-16-day">Weatherbit | 16 Day Forecast</a>
* Acessing 7 days of forecast data in json format.
* Data requests
    1. temperature (Celsius)
    2. humidity (relative humidity in %)
    3. rainfall (mm)
* API call format: ```https://api.weatherbit.io/v2.0/forecast/daily?city=Raleigh,NC&key=API_KEY&days=5```
* Effective Rainfall: https://www.fao.org/3/S2022E/s2022e03.htm
* Rates of rainfall:
    * Drizzle, very small droplets.
    * Slight (fine) drizzle: Detectable as droplets only on the face, car windscreens and windows.
    * Moderate drizzle: Windows and other surfaces stream with water.
    * Heavy (thick) drizzle: Impairs visibility and is measurable in a raingauge, rates up to 1 mm per hour.
    * Rain, drops of appreciable size and may be described as small to large drops. It is possible to have rain drops within drizzle!
    * Slight rain: Less than 0.5 mm per hour.
    * Moderate rain: Greater than 0.5 mm per hour, but less than 4.0 mm per hour.
    * Heavy rain: Greater than 4 mm per hour, but less than 8 mm per hour.
    * Very heavy rain: Greater than 8 mm per hour.
    * Slight shower: Less than 2 mm per hour.
    * Moderate shower: Greater than 2 mm, but less than 10 mm per hour.
    * Heavy shower: Greater than 10 mm per hour, but less than 50 mm per hour.
    * Violent shower: Greater than 50 mm per hour.
* Weather API Codes/Icons: https://www.weatherbit.io/api/codes 


### Fertilization 
* Heavy and prolonged rain can wash away recently applied fertilizer. It is also important that the water within the first 24 hours (upto 48 hours) be evenly distributed.
* Rainfall of greater than ½ inch (12.7 mm) puts your fertilizer at risk. 
* Urea converts to ammonium ion within 5 to six days by urease enzyme and it takes another 5 to. Six days to convert it in nitrate by soil bacteria . After this plant roots can absorb it properly but all these are also affected by temp, moisture and soil texture.

### Version Info
* Python version: 3.8.10
* requests module version: 2.27.1
* json module version: 2.0.9

In [5]:
import requests as rq
import json as js
from time import sleep

# !python3 --version
# print("requests", rq.__version__)
# print("json", js.__version__)

In [6]:
class BestTimeToFertilize:
    __BASE_URL = "https://api.weatherbit.io/v2.0/forecast/daily?"
    __API_KEY = "480589e42e7c4352abe4fe25bd398ab0"
    
    
    def __init__(self, city_name = 'Bangalore', state_name = 'Karnataka', days = 7):
        self.city_name = '+'.join(city_name.lower().strip().split())
        self.state_name = '+'.join(state_name.lower().strip().split())
        self.country_name = 'IN'
        self.days = days
        self.response = None
        self.response_code = None
        self.weather_data = list()
        
    def api_caller(self):
        try:
            complete_url = "{0}city={1}&state={2}&country={3}&key={4}&days={5}".format(self.__BASE_URL, self.city_name, self.state_name, self.country_name, self.__API_KEY, self.days)
            print(complete_url)
            #while self.response == None:
            self.response = rq.get(complete_url)
            sleep(5)
            self.response_code = self.response.status_code
            return self.response_code
        except Exception as msg:
            print("api_caller():", msg)
            return -1
        
    
    def is_api_call_success(self):
        if self.response_code == 200:
            return True
        elif self.response_code == 204:
            print('Content Not available, error code 204')
        return False
    

    def json_file_bulider(self):
        try:
            json_obj = self.response.json()
            with open('weather_data.json', 'w') as file:
                js.dump(json_obj, file, indent = 1, sort_keys = True)
            print("weather_data.json file build successfully")
        except Exception as msg:
            print("json_bulider():", msg)
            
    
    def best_time_fertilize(self):
        json_obj = self.response.json()
        
        print("City:", json_obj['city_name'], "\n")

        prolonged_precip = 0
        prolonged_prob = 0
        heavy_rain_2d = False
        heavy_rain_chance_2d = 0
        precip_2d = 0
        precip_chance_2d = 0
        
        for i in range(self.days):
            date = json_obj['data'][i]['datetime']
            temp = json_obj['data'][i]['temp']
            rh = json_obj['data'][i]['rh']
            precip = json_obj['data'][i]['precip']
            prob = json_obj['data'][i]['pop']
            w_code = json_obj['data'][i]['weather']['code']
            w_desc = json_obj['data'][i]['weather']['description']
            i_code = json_obj['data'][i]['weather']['icon']
            prolonged_precip += precip
            prolonged_prob += prob

            count_2d = 0
            if i < 2:
                precip_2d += precip
                precip_chance_2d += prob
                if w_code in [202, 233, 502, 521, 522]:
                    heavy_rain_2d = True
                    heavy_rain_chance_2d += prob
                    count_2d += 1
                    heavy_rain_chance_2d //= count_2d
            
            di = {
                  "Date": str(date), 
                  "Temperature": str(temp), 
                  "Relative Humidity": str(rh), 
                  "Rainfall": str(precip), 
                  "Probability of Precipitation": str(prob),
                  "Weather code": str(w_code),
                  "Weather Description": str(w_desc),
                  "Icon code": str(i_code)
                 }
            self.weather_data.append(di)
            
            print("Date:", date)
            print("Temperature:", temp)
            print("Relative Humidity:", rh)
            print("Rainfall:", precip)
            print("Probability of Precipitation:", prob)
            print("Weather code:", w_code, "->", w_desc)
            print()

        prolonged_prob //= self.days
        precip_chance_2d //= 2

        print("-"*80)
        print("The amount of rain for 2 days, counting today:", precip_2d)
        print("Chances of rain for 2 days, counting today:", precip_chance_2d)
        print()

        if heavy_rain_2d:
            print("*"*21, "Warning !!!", "*"*21)
            print("Heavy Rain Chances:", heavy_rain_chance_2d)
            print("Heavy Rainfall puts your fertilizer at risk.")
            print("*"*21, "Warning !!!", "*"*21)

        print("Prolonged Precipitation:", prolonged_precip)
        print("Prolonged Precipitation Probability:", prolonged_prob)
        print()

        if prolonged_precip > 12.7 and prolonged_prob >= 50:
            print("*"*21, "Warning !!!", "*"*21)
            print("Prolonged Rainfall of greater than 12.7 mm puts your fertilizer at risk.")
            print("*"*21, "Warning !!!", "*"*21)

In [7]:
if __name__ == '__main__':
    obj = BestTimeToFertilize('Ahmedabad', 'Gujarat')
    print(obj.api_caller())
    if obj.is_api_call_success():
        obj.best_time_fertilize()
        obj.json_file_bulider()
#         print(obj.weather_data)

https://api.weatherbit.io/v2.0/forecast/daily?city=ahmedabad&state=gujarat&country=IN&key=480589e42e7c4352abe4fe25bd398ab0&days=7
403


## Predictive Module

### Version Info
* numpy 1.22.3
* pandas 1.3.3

In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import warnings

# enable the inline plotting, where the plots/graphs will be displayed just below the 
# cell where your plotting commands are written
%matplotlib inline

warnings.filterwarnings('ignore')

# print("numpy", np.__version__)
# print("pandas", pd.__version__)

ModuleNotFoundError: No module named 'numpy'

In [ ]:
data = 'Nutrient_recommendation.csv'
df = pd.read_csv(data, header=None)

In [ ]:
# view dimensions of dataset

df.shape

In [ ]:
# preview the dataset

df.head()

In [ ]:
# Rename column names

df.columns = ['Crop', 'Temperature', 'Humidity', 'Rainfall', 'Label_N', 'Label_P', 'Label_K']
df.drop(df.index[:1], inplace=True)
df.head()

In [ ]:
# View summary of dataset

df.info()

In [ ]:
# check missing values in variables

df.isnull().sum()

In [ ]:
# Declare feature vector and target variable
X = df.drop(['Label_N', 'Label_P', 'Label_K'], axis=1)

y1 = df['Label_N']
y2 = df['Label_P']
y3 = df['Label_K']

In [ ]:
# split data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size = 0.20, random_state = 42)

In [ ]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape

In [ ]:
X_train.head()

In [ ]:
# create mapping of crop(string) to int type
mapping = dict()

# writing the mapping in a file
with open("mapped_crops.csv", "w") as fh:
    fh.write("Crops,Key\n")
    for i, crop in enumerate(np.unique(df[['Crop']]), 1):
        mapping[crop] =  i
        fh.write("%s,%d\n" % (crop, i))
    mapping['NA'] = np.nan
    fh.write("NA,nan")
    
print(mapping)

In [ ]:
# import category encoders
import category_encoders as ce

ordinal_cols_mapping = [{"col": "Crop", "mapping": mapping}, ]

# encode categorical variables with ordinal encoding
encoder = ce.OrdinalEncoder(cols = 'Crop', mapping = ordinal_cols_mapping, return_df = True)

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)
X_train.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
 
# n_estimators = The number of trees in the forest.
regressor = RandomForestRegressor(n_estimators = 50, random_state = 0)
 
# fit the model
regressor.fit(X_train, y_train)

# Predict the Test set results
y_pred = regressor.predict(X_test)

### Calculating the loss after training
* Let us now calculate the loss between the actual target values in the testing set and the values predicted by the model with the use of a cost function called the Root Mean Square Error (RMSE).
<img src="https://miro.medium.com/max/460/0*rJwdJuCjM3fqfCIM.png">
* The RMSE of a model determines the absolute fit of the model to the data. In other words, it indicates how close the actual data points are to the model’s predicted values. A low value of RMSE indicates a better fit and is a good measure for determining the accuracy of the model’s predictions.

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

rmse = float(format(np.sqrt(metrics.mean_squared_error(y_test, y_pred)), '.3f'))
print("\nRMSE: ", rmse)

In [ ]:
# If set to -1, all CPUs are used, This parameter is used to specify how many concurrent 
# processes or threads should be used for routines that are parallelized

model = RandomForestRegressor(n_jobs=-1)

In [ ]:
estimators = np.arange(10, 200, 10)
scores = []
for n in estimators:
    model.set_params(n_estimators=n)
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))
plt.title("Effect of n_estimators")
plt.xlabel("n_estimator")
plt.ylabel("score")
plt.plot(estimators, scores)

### Accuracy
* N: 0.871
* P: 0.926
* K: 0.995

In [ ]:
scores_arr = [round(sc, 3) for sc in scores]
unique, counts = np.unique(scores_arr, return_counts = True)

max_count = max(counts)
accuracy = -1
for uni, count in zip(unique, counts):
    # print(uni, count)
    if count == max_count:
        accuracy = uni

print("Model accuracy: %.3f" % (accuracy))

# NPKEstimator

In [ ]:
import warnings
import numpy as np 
import pandas as pd 
from sklearn import metrics
import category_encoders as ce
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

warnings.filterwarnings('ignore')

In [ ]:
class NPKEstimator:
    def __init__(self, data = 'Nutrient_recommendation.csv', ):
        self.df = pd.read_csv(data, header=None)
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        
    
    def renameCol(self):
        self.df.columns = ['Crop', 'Temperature', 'Humidity', 'Rainfall', 'Label_N', 'Label_P', 'Label_K']
        self.df.drop(self.df.index[:1], inplace=True)
    
    
    def cropMapper(self):
        # create mapping of crop(string) to int type
        mapping = dict()

        with open("mapped_crops.csv", "w") as fh:
            fh.write("Crops,Key\n")
            for i, crop in enumerate(np.unique(self.df[['Crop']]), 1):
                mapping[crop] =  i
                fh.write("%s,%d\n" % (crop, i))
            mapping['NA'] = np.nan
            fh.write("NA,nan")
        # print(mapping)
        
        ordinal_cols_mapping = [{"col": "Crop", "mapping": mapping}, ]
        encoder = ce.OrdinalEncoder(cols = 'Crop', mapping = ordinal_cols_mapping, return_df = True)
        return mapping, encoder
    
    
    def estimator(self, crop, temp, humidity, rainfall, y_label):
        X = self.df.drop(['Label_N', 'Label_P', 'Label_K'], axis=1)
        y = self.df[y_label]
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
        
        mapping, encoder = self.cropMapper()
        self.X_train = encoder.fit_transform(self.X_train)
        self.X_test = encoder.transform(self.X_test)
        
        regressor = RandomForestRegressor(n_estimators = 50, random_state = 0)
        regressor.fit(self.X_train, self.y_train)
        
        # y_pred = regressor.predict(self.X_test)
        query = [mapping[crop.strip().lower()], temp, humidity, rainfall]
        y_pred = regressor.predict([query])
        return y_pred
    
    
    def accuracyCalculator(self):
        model = RandomForestRegressor(n_jobs=-1)
        estimators = np.arange(10, 200, 10)
        scores = []
        for n in estimators:
            model.set_params(n_estimators=n)
            model.fit(self.X_train, self.y_train)
            scores.append(model.score(self.X_test, self.y_test))
        
        scores_arr = [round(sc, 3) for sc in scores]
        unique, counts = np.unique(scores_arr, return_counts = True)

        max_count = max(counts)
        accuracy = -1
        for uni, count in zip(unique, counts):
            # print(uni, count)
            if count == max_count:
                accuracy = uni

        # print("Model accuracy: %.2f" % (accuracy))
        return accuracy

In [ ]:
if __name__ == '__main__':
    obj = NPKEstimator()
    obj.renameCol()
    # 'Label_N', 'Label_P', 'Label_K'
    # rice,21.94766735,80.97384195,213.3560921,67,59,41
    crop, temp, humidity, rainfall, y_label = 'rice',21.94766735,80.97384195,213.3560921,'Label_N'
    res = obj.estimator(crop, temp, humidity, rainfall, y_label)
    print(y_label, ":", res[0])